# Texto a vectores (Lemma)

**Descripción :**

Una vez que ya tenemos nuestros datos limpios, lo que procede es convertirlos a vectores para poder utilizarlos en nuestro modelo de aprendizaje. Utilizaremos word2vec para convertir nuestros textos a vectores y también haremos la matriz TF-IDF para poder comparar los resultados.

Word2Vec form Google : https://code.google.com/archive/p/word2vec/

**Resources:**
* https://medium.com/swlh/word-embedding-word2vec-with-genism-nltk-and-t-sne-visualization-43eae8ab3e2e
* https://radimrehurek.com/gensim/models/word2vec.html

**Team:**  
* Chaparro Sicardo Tanibeth  
* Malváez Flores Axel Daniel

In [1]:
# Librerías que necesitaremos para el procedimiento
import numpy as np
import pandas as pd
import re
from gensim.models import word2vec, Word2Vec, KeyedVectors
from utils import *
%matplotlib inline
pd.options.mode.chained_assignment = None

## Preprocessing data for Lemmatization

* Realizamos el mismo proceso para nuestro texto lematizado

In [2]:
# Cargamos los datos
data = pd.read_csv('../Data/lemma_df.csv')

# Regex para eliminar los corchetes del string
regex = re.compile(r'[\[\]]')

# Aplicamos la regex a la columna 'stemming_description' y hacemos un split sobre la coma
data['lemma_description'] = data['lemma_description'].apply(lambda x: regex.sub('', x).split(', '))

# Creamos el corpus
corpus = data['lemma_description']
corpus = corpus.tolist()

In [3]:
############ Skip-gram
model_sg_lemma = Word2Vec(corpus, window=20, min_count=2, workers=4, vector_size=300, sg=1)
model_sg_lemma.save('../Data/Word2Vec-Own/word2vecSGLemma.model')

word_vectors_sg_lemma = model_sg_lemma.wv
word_vectors_sg_lemma.save('../Data/Word2Vec-Own/word2vecSGLemma.wordvectors')

del model_sg_lemma

########### CBOW
model_cbow_lemma = Word2Vec(corpus, window=20, min_count=2, workers=4, vector_size=300, sg=0)
model_cbow_lemma.save('../Data/Word2Vec-Own/word2vecCBOWLemma.model')

word_vectors_cbow_lemma = model_cbow_lemma.wv
word_vectors_cbow_lemma.save('../Data/Word2Vec-Own/word2vecCBLemma.wordvectors')

del model_cbow_lemma

In [4]:
# Load back with memory-mapping = read-only, shared across processes.
wv = KeyedVectors.load('../Data/Word2Vec-Own/word2vecSGLemma.wordvectors', mmap='r')

# Generate document vectors for each document in your corpus
size = wv.vector_size
document_vectors = np.array([document_vectorizer(doc_token, wv, vector_size=size) for doc_token in corpus])

print('Dimensión de nuestra matriz de documentos (ya vectorizados):', document_vectors.shape)

Dimensión de nuestra matriz de documentos (ya vectorizados): (50424, 300)


### TF-IDF matrix

In [7]:
from sklearn.feature_extraction.text import TfidfVectorizer

docs = []

for doc in data['lemma_description']:
    doc_join = ' '.join(doc)
    docs.append(doc_join)

# Crear un objeto TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer()

tfidf_matrix = tfidf_vectorizer.fit_transform(docs)
tfidf_matrix_array = tfidf_matrix.toarray()

### Google word2vec

In [ ]:
# Load the pre-trained Word2Vec model
model_google = KeyedVectors.load_word2vec_format('../Data/Word2Vec-Google/GoogleNews-vectors-negative300.bin', binary=True)
vectors = model_google.vectors

In [ ]:
document_vectors_google = np.array([document_vectorizer(doc_token, vectors) for doc_token in corpus])
print('Dimensión de nuestra matriz de documentos (ya vectorizados con Google):', document_vectors_google.shape)

## Classification

Loading...